In [1]:
import chess
import chess.pgn
import random
import numpy as np
from IPython.display import display
import time
import tensorflow as tf
import os
import datetime
import copy
import re

from chess_functions import split_dims,MoveTensorLegal,TopN

In [2]:
switcher = {
    (1,0):0,(2,0):1,(3,0):2,(4,0):3,(5,0):4,(6,0):5,(7,0):6,(-1,0):7,(-2,0):8,(-3,0):9,(-4,0):10,(-5,0):11,(-6,0):12,(-7,0):13,(0,1):14,(0,2):15,(0,3):16,(0,4):17,(0,5):18,(0,6):19,(0,7):20,(0,-1):21,(0,-2):22,(0,-3):23,(0,-4):24,(0,-5):25,(0,-6):26,(0,-7):27,(1,1):28,(2,2):29,(3,3):30,(4,4):31,(5,5):32,(6,6):33,(7,7):34,(-1,-1):35,(-2,-2):36,(-3,-3):37,(-4,-4):38,(-5,-5):39,(-6,-6):40,(-7,-7):41,(1,-1):42,(2,-2):43,(3,-3):44,(4,-4):45,(5,-5):46,(6,-6):47,(7,-7):48,(-1,1):49,(-2,2):50,(-3,3):51,(-4,4):52,(-5,5):53,(-6,6):54,(-7,7):55,(2,1):56,(2,-1):57,(-2,1):58,(-2,-1):59,(1,2):60,(-1,2):61,(1,-2):62,(-1,-2):63,
        
    }

dicV = []

for a in range(8):
    for b in range(8):  
        for tup, value in switcher.items():
            y=a+tup[0]
            x=b+tup[1]
            if 0<=x<8 and 0<=y<8:
                dicV.append(value+b*64+a*8*64)

In [3]:
k=3

BinaryLegalVector = tf.keras.Input(shape=(1792))
Input_2d = tf.keras.Input(shape=(8, 8, 72))
Input_1d = tf.keras.Input(shape=(34))

f = tf.keras.layers.Dense(8*8*128)(Input_1d)
f = tf.reshape(f, [-1, 8, 8, 128])

x1 = tf.keras.layers.Conv2D(128 , (5,5), padding='same')(Input_2d)
x2 = tf.keras.layers.Conv2D(128 , (k,k),dilation_rate=3, padding='same')(Input_2d)
x3 = tf.keras.layers.Conv2D(128 , (k,k),dilation_rate=4, padding='same')(Input_2d)
x4 = tf.keras.layers.Conv2D(128 , (k,k),dilation_rate=5, padding='same')(Input_2d)
x5 = tf.keras.layers.Conv2D(128 , (k,k),dilation_rate=6, padding='same')(Input_2d)
x6 = tf.keras.layers.Conv2D(128 , (k,k),dilation_rate=7, padding='same')(Input_2d)
x = tf.keras.layers.Add()([x1, x2, x3, x4, x5, x6])

x = x + f
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation(tf.keras.activations.relu)(x)

for i in range(6):
    p = x
    
    x1 = tf.keras.layers.Conv2D(128 , (5,5), padding='same')(x)
    x2 = tf.keras.layers.Conv2D(128 , (k,k),dilation_rate=3, padding='same')(x)
    x3 = tf.keras.layers.Conv2D(128 , (k,k),dilation_rate=4, padding='same')(x)
    x4 = tf.keras.layers.Conv2D(128 , (k,k),dilation_rate=5, padding='same')(x)
    x5 = tf.keras.layers.Conv2D(128 , (k,k),dilation_rate=6, padding='same')(x)
    x6 = tf.keras.layers.Conv2D(128 , (k,k),dilation_rate=7, padding='same')(x)
    x = tf.keras.layers.Add()([x1, x2, x3, x4, x5, x6])
    
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation(tf.keras.activations.relu)(x)
    
    x1 = tf.keras.layers.Conv2D(128 , (5,5), padding='same')(x)
    x2 = tf.keras.layers.Conv2D(128 , (k,k),dilation_rate=3, padding='same')(x)
    x3 = tf.keras.layers.Conv2D(128 , (k,k),dilation_rate=4, padding='same')(x)
    x4 = tf.keras.layers.Conv2D(128 , (k,k),dilation_rate=5, padding='same')(x)
    x5 = tf.keras.layers.Conv2D(128 , (k,k),dilation_rate=6, padding='same')(x)
    x6 = tf.keras.layers.Conv2D(128 , (k,k),dilation_rate=7, padding='same')(x)
    x = tf.keras.layers.Add()([x1, x2, x3, x4, x5, x6])
    x = tf.keras.layers.BatchNormalization()(x)
    
    q = x
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(40, activation='relu')(x)
    x = tf.keras.layers.Dense(128, activation='sigmoid')(x)
    x = tf.keras.layers.multiply([x, q])
    
    x = tf.keras.layers.Add()([x,p])
    x = tf.keras.layers.Activation(tf.keras.activations.relu)(x)

x1 = tf.keras.layers.Conv2D(128 , (5,5), padding='same')(x)
x2 = tf.keras.layers.Conv2D(128 , (k,k),dilation_rate=3, padding='same')(x)
x3 = tf.keras.layers.Conv2D(128 , (k,k),dilation_rate=4, padding='same')(x)
x4 = tf.keras.layers.Conv2D(128 , (k,k),dilation_rate=5, padding='same')(x)
x5 = tf.keras.layers.Conv2D(128 , (k,k),dilation_rate=6, padding='same')(x)
x6 = tf.keras.layers.Conv2D(128 , (k,k),dilation_rate=7, padding='same')(x)
x = tf.keras.layers.Add()([x1, x2, x3, x4, x5, x6])
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation(tf.keras.activations.relu)(x)

x = tf.keras.layers.Conv2D(64, (1,1), padding='same')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation(tf.keras.activations.relu)(x)

x = tf.reshape(x, [-1, 8*8*64])
x = tf.gather(x, dicV, axis=-1)
x = tf.keras.layers.Add()([BinaryLegalVector, x])
x = tf.keras.layers.Softmax()(x)

model = tf.keras.Model([BinaryLegalVector, Input_2d, Input_1d], x)

2022-08-13 07:20:17.661737: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-08-13 07:20:18.134325: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22309 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:c1:00.0, compute capability: 8.6


In [4]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 8, 8, 72)]   0           []                               
                                                                                                  
 input_3 (InputLayer)           [(None, 34)]         0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 8, 8, 128)    230528      ['input_2[0][0]']                
                                                                                                  
 conv2d_1 (Conv2D)              (None, 8, 8, 128)    83072       ['input_2[0][0]']                
                                                                                              

                                                                                                  
 global_average_pooling2d (Glob  (None, 128)         0           ['batch_normalization_2[0][0]']  
 alAveragePooling2D)                                                                              
                                                                                                  
 dense_1 (Dense)                (None, 40)           5160        ['global_average_pooling2d[0][0]'
                                                                 ]                                
                                                                                                  
 dense_2 (Dense)                (None, 128)          5248        ['dense_1[0][0]']                
                                                                                                  
 multiply (Multiply)            (None, 8, 8, 128)    0           ['dense_2[0][0]',                
          

 conv2d_32 (Conv2D)             (None, 8, 8, 128)    147584      ['activation_4[0][0]']           
                                                                                                  
 conv2d_33 (Conv2D)             (None, 8, 8, 128)    147584      ['activation_4[0][0]']           
                                                                                                  
 conv2d_34 (Conv2D)             (None, 8, 8, 128)    147584      ['activation_4[0][0]']           
                                                                                                  
 conv2d_35 (Conv2D)             (None, 8, 8, 128)    147584      ['activation_4[0][0]']           
                                                                                                  
 add_7 (Add)                    (None, 8, 8, 128)    0           ['conv2d_30[0][0]',              
                                                                  'conv2d_31[0][0]',              
          

                                                                                                  
 conv2d_49 (Conv2D)             (None, 8, 8, 128)    147584      ['activation_7[0][0]']           
                                                                                                  
 conv2d_50 (Conv2D)             (None, 8, 8, 128)    147584      ['activation_7[0][0]']           
                                                                                                  
 conv2d_51 (Conv2D)             (None, 8, 8, 128)    147584      ['activation_7[0][0]']           
                                                                                                  
 conv2d_52 (Conv2D)             (None, 8, 8, 128)    147584      ['activation_7[0][0]']           
                                                                                                  
 conv2d_53 (Conv2D)             (None, 8, 8, 128)    147584      ['activation_7[0][0]']           
          

 global_average_pooling2d_4 (Gl  (None, 128)         0           ['batch_normalization_10[0][0]'] 
 obalAveragePooling2D)                                                                            
                                                                                                  
 dense_9 (Dense)                (None, 40)           5160        ['global_average_pooling2d_4[0][0
                                                                 ]']                              
                                                                                                  
 dense_10 (Dense)               (None, 128)          5248        ['dense_9[0][0]']                
                                                                                                  
 multiply_4 (Multiply)          (None, 8, 8, 128)    0           ['dense_10[0][0]',               
                                                                  'batch_normalization_10[0][0]'] 
          

                                                                                                  
 conv2d_81 (Conv2D)             (None, 8, 8, 128)    147584      ['activation_12[0][0]']          
                                                                                                  
 conv2d_82 (Conv2D)             (None, 8, 8, 128)    147584      ['activation_12[0][0]']          
                                                                                                  
 conv2d_83 (Conv2D)             (None, 8, 8, 128)    147584      ['activation_12[0][0]']          
                                                                                                  
 add_19 (Add)                   (None, 8, 8, 128)    0           ['conv2d_78[0][0]',              
                                                                  'conv2d_79[0][0]',              
                                                                  'conv2d_80[0][0]',              
          

In [5]:
def testgen():
    
    currfileint = start_file_num
    arch=np.load(str(source_directory)+str(currfileint-1)+'.npz')#, allow_pickle=True)
    
    str1 = str("File index %d loaded" % (currfileint-1))
    print(str1)
    
    if 'file_object' in globals():
        file_object.write("\n"+str1)
    
    intern = 0 
    epoch=initial_epoch_int
    
    x_bool  = arch['arr_0']
    x_float = arch['arr_1']
    b       = arch['arr_2']
    y       = arch['arr_3']
    t       = arch['arr_4']
    
    for i in range(num_training_samples):
        if (intern+1) > source_pos_per_file:

            if currfileint==(num_training_samples/source_pos_per_file):
                currfileint=1
            else:
                currfileint+=1
            
            arch=np.load(str(source_directory)+str(currfileint-1)+'.npz')#, allow_pickle=True)            
            intern=0

            x_bool  = arch['arr_0']
            x_float = arch['arr_1']
            b       = arch['arr_2']
            y       = arch['arr_3']
            t       = arch['arr_4']
            
            str2 = str("File index %d loaded" % (currfileint-1))
            print(str2)
            
            if 'file_object' in globals():
                file_object.write("\n"+str2)
  

        batch_x_bool  = x_bool[intern]
        batch_x_float = x_float[intern]
        batch_b       = b[intern]
        batch_y       = y[intern]
        batch_t       = t[intern]

        intern+=1           

        yield {"input_1": batch_x_bool, "input_2": batch_x_float, "input_3": batch_b}, {"input_1": batch_y, "input_2": batch_t} #, "input_3": batch_o


In [6]:
batch_size=512
source_pos_per_file=65536 #32768
num_training_samples=65536*1299
test_length = 0
#testfile=np.load("E1299.npz")

source_directory='D'
initial_epoch_int=1
start_file_num=1

epochs=500
logdir = 'log.txt'
log = False
inistep=0
avrgs=[100,500,1000]

In [ ]:
cce_loss = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(5e-3)

dataset = tf.data.Dataset.from_generator(testgen,        
output_types=({"input_1": tf.bool,
               "input_2": tf.float16,
               "input_3": tf.bool},
              {"input_1": tf.bool,
               "input_2": tf.float16}))

dataset = dataset.batch(batch_size).prefetch(6)

t0=time.time()
summ = []

for epoch in range(epochs):    
    
    str4 = str("\nStart of epoch %d" % (epoch,))
    print(str4)
    
    if log:
        file_object.write("\n"+str4)

    for step, (xxb, yt) in enumerate(dataset):
        
        x_b = xxb['input_1']
        x_f = xxb['input_2']
        b   = xxb['input_3']
        y   = yt['input_1']
        #t = yt['input_2']
        
        b = tf.cast(b, dtype=tf.float32)
        b = -tf.float32.max * b
        
        with tf.GradientTape() as tape:
            logits = model([b, x_b, x_f], training=True) 
            loss_cce = cce_loss(y, logits)
  
            loss_value = loss_cce
                     
            
        grads = tape.gradient(loss_value, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))

        summ.append(loss_value)        

        if step % 10 == 0:
            str1 = str("Batch loss at step %d: %.4f %.2fsec lr:%.6f" % (step+inistep, float(loss_value), time.time()-t0, optimizer.lr))
            print(str1)
            
            if log:
                file_object.write("\n"+str1)
            #file_object.write("\n")
            
        for i in avrgs:
            if step % i == 0:
                str2 = str("Mean loss len="+str(i)+": %.4f" % ((sum(summ[-i:])/len(summ[-i:]))))
                print(str2)

                if log:
                    file_object.write("\n"+str2)
                    
        summ = summ[-1000:]
            
        t0=time.time()
    
        if test_length>0 and step % 1000 == 0:
            val = TopN(testfile,test_length,model,1)

            str3 = 'Move prediction accuracy:'+"{:.3%}".format(val)
            print(str3)
            #str4 = 'Time taken to move error:'+"{:.3}s".format(time_stuff)
            #print(str4) 
            
            if log:
                file_object.write("\n"+str3)
                #file_object.write("\n"+str4)

        if step+inistep==2000:
            optimizer.lr=3e-3

        if step+inistep==3200:
            optimizer.lr=1.5e-3          
            
        if step+inistep==4200:
            optimizer.lr=8e-4
        
        if step+inistep==10_000:
            optimizer.lr=5e-4
            
        if step+inistep==25_000:
            optimizer.lr=1e-4
        

        if step % 5_000 == 0:
            if log:
                file_object.write("\n"+'model_2_'+str(step+inistep)+'.h5')
                file_object.close()
                file_object = open(logdir, 'a')
                
            model.save('model_2_'+str(step+inistep)+'.h5')